In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Dropout, MaxPool1D, ReLU, Flatten

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
samples_per_class = 5000
no_of_classes = 2

In [ ]:
noise_df = pd.read_csv("/content/gdrive/MyDrive/GW_SOP/Code/debugging/1D-CNN Binary Classification/Signal_VISIBLE_in_strain_data/Signal_NOT_WHITENED/Test 3/data/Final_Merged_Noise_Reduced_No_Abs.csv", header=None)
noise = noise_df.values.astype(float)
noise = np.repeat(noise, 5000, 0)


data_BBH_df = pd.read_csv("/content/gdrive/MyDrive/GW_SOP/Code/debugging/1D-CNN Binary Classification/Signal_VISIBLE_in_strain_data/Signal_NOT_WHITENED/Test 3/data/Final_BBH_Merged_Noise_Signal.csv", header=None)
data_BBH = data_BBH_df.values.astype(float)

In [ ]:
X = np.concatenate((noise, data_BBH), axis=0)
X = X[:10000, :]
print(X.shape)

In [ ]:
y = [int(i/samples_per_class) for i in range(samples_per_class*no_of_classes)]
y = tf.keras.utils.to_categorical(y)
print(y.shape)

In [ ]:
# scaler = StandardScaler()
# X = scaler.fit_transform(X)
# print(X)

X *= 1e19
print(X)

In [ ]:
X = np.expand_dims(X, axis=-1)
print(X.shape)

In [ ]:
def create_model():    
    model = Sequential()

    model.add(Conv1D(16, 16, input_shape = (16384,1)))
    model.add(MaxPool1D(4, 4))
    model.add(ReLU())
    model.add(Conv1D(32, 8))
    model.add(MaxPool1D(4, 4))
    model.add(ReLU())
    model.add(Conv1D(64, 8))
    model.add(MaxPool1D(4, 4))
    model.add(ReLU())
    model.add(Conv1D(128, 8))
    model.add(MaxPool1D(4, 4))
    model.add(ReLU())
    model.add(Flatten())
    model.add(Dense(7808))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))

    model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
N_splits = 5
n_epochs = 2

kf = KFold(n_splits = N_splits, shuffle = True)
acc = [0 for i in range(n_epochs+1)]
loss = [0 for i in range(n_epochs)]
precision = 0
recall = 0
score = 0
cm = np.zeros((2,2))

for train_index, test_index in kf.split(X):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    # print(y_train)
    # print(y_test)

    model = create_model()
    history = model.fit(X_train, y_train, batch_size=128, epochs=n_epochs)

    print("---------------------------------------------------------------------------------------------------------")

    tmp_acc = history.history['accuracy']
    tmp_loss = history.history['loss']

    for i in range(n_epochs):
        acc[i+1] += tmp_acc[i]
        loss[i] += tmp_loss[i]

    pred = model.predict(X_test)
    y_pred = np.argmax(pred, axis = 1)
    y_true = np.argmax(y_test, axis = 1)

    precision += precision_score(y_true, y_pred, average='binary')
    recall += recall_score(y_true, y_pred, average='binary')
    score += f1_score(y_true, y_pred, average='binary')
    cm = np.add(cm, confusion_matrix(y_true, y_pred, labels=[0, 1]))

In [ ]:
final_accuracy = acc[n_epochs]/N_splits
final_precision = precision/N_splits
final_recall = recall/N_splits
final_score = score/N_splits
final_cm = cm/N_splits

print('Accuracy: %.3f' % final_accuracy)
print('Precision: %.3f' % final_precision)
print('Recall: %.3f' % final_recall)
print('F1-Score: %.3f' % final_score)

disp = ConfusionMatrixDisplay(confusion_matrix=final_cm, display_labels=[0, 1])
disp.plot()
plt.show()

In [ ]:
acc_1 = [acc[i]/N_splits for i in range(n_epochs+1)]
loss_1 = [loss[i]/N_splits for i in range(n_epochs)]

plt.figure(figsize=(13, 4))
plt.subplot(1, 2, 1)
plt.plot(acc_1, label='Training Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training Accuracy')
plt.xlabel('epoch')

plt.subplot(1, 2, 2)
plt.plot(loss_1, label='Training Loss')
plt.legend(loc='upper right')
plt.ylabel('Categorical Cross Entropy')
plt.ylim([0,2])
plt.title('Training Loss')
plt.xlabel('epoch')
plt.show()